### Introns data prepared

In [ ]:
import pandas as pd
import pysam
from collections import defaultdict

merged_file = "data/RNA_Proteomics_Merged.csv"
gtf_file = "/mnt/data_2/ensembl/Homo_sapiens.GRCh38.113.gtf"
fasta_file = "/mnt/data_3/hallep/reference/hg38.fa"
out_fasta = "data/introns_9056.fa"
out_csv   = "data/introns_9056_metadata.csv"

merged_file = pd.read_csv(merged_file)
merged_file["ENSG"] = merged_file["Unnamed: 0"].str.replace(r"\.\d+$", "", regex=True)

# load ENSG IDs from merged dataset
ensg_ids = set(merged_file["ENSG"].dropna())

# parse GTF to collect exons by transcript 
exons_by_tx = defaultdict(list)

with open(gtf_file) as f:
    for line in f:
        if line.startswith("#"):
            continue
        chrom, source, feature, start, end, score, strand, frame, attrs = line.strip().split("\t")
        if feature != "exon":
            continue
        # parse attributes
        attr_dict = {kv.split(" ")[0]: kv.split(" ")[1].strip('"') 
                     for kv in attrs.split(";") if kv.strip()}
        gene_id = attr_dict.get("gene_id")
        transcript_id = attr_dict.get("transcript_id")
        if gene_id in ensg_ids:
            exons_by_tx[(gene_id, transcript_id, chrom, strand)].append((int(start), int(end)))

# Compute introns from exon coordinates
introns = []
for (gene_id, tx, chrom, strand), exons in exons_by_tx.items():
    exons_sorted = sorted(exons, key=lambda x: x[0])
    for i in range(len(exons_sorted)-1):
        intron_start = exons_sorted[i][1] + 1
        intron_end = exons_sorted[i+1][0] - 1
        if intron_start < intron_end:
            intron_num = i + 1  
            introns.append((gene_id, tx, chrom, strand, intron_start, intron_end, intron_num))



# Extract sequences from FASTA 
fasta = pysam.FastaFile(fasta_file)
all_refs = set(fasta.references)

def normalize_chrom(chrom):
    """Fix chromosome name mismatches between GTF and FASTA"""
    if chrom in all_refs:
        return chrom
    if "chr" + chrom in all_refs:
        return "chr" + chrom
    if chrom.startswith("chr") and chrom[3:] in all_refs:
        return chrom[3:]
    return None

with open(out_fasta, "w") as out_fa, open(out_csv, "w") as out_tab:
    out_tab.write("gene_id,transcript_id,intron_num,chrom,start,end,strand,length\n")
    for gene_id, tx, chrom, strand, s, e, intron_num in introns:
        chrom_name = normalize_chrom(chrom)
        if chrom_name is None:
            continue

        try:
            seq = fasta.fetch(chrom_name, s-1, e).upper()  
        except Exception as ex:
            print(f" Could not fetch {chrom_name}:{s}-{e} for {gene_id}: {ex}")
            continue

        if strand == "-":  # reverse complement
            seq = seq.translate(str.maketrans("ACGT", "TGCA"))[::-1]

        header = f">{gene_id}|{tx}|intron{intron_num}|{chrom}:{s}-{e}({strand})"
        out_fa.write(f"{header}\n{seq}\n")

        out_tab.write(f"{gene_id},{tx},{intron_num},{chrom_name},{s},{e},{strand},{len(seq)}\n")



### STEP1: Length

In [ ]:
# Length for codon97

import pandas as pd

# Load intron metadata
intron_df = pd.read_csv("data/introns_9056_metadata.csv")

# Collapse to per-gene total intron length
intron_total = (
    intron_df.groupby("gene_id")["length"]
    .sum()
    .reset_index()
    .rename(columns={"length": "total_intron_length"})
)

# Load Codon97 dataset
codon97_df = pd.read_csv("data/RNA_Proteomics_Filtered.csv")

# Merge on ENSG
merged = codon97_df.merge(
    intron_total, 
    left_on="ENSG_clean", 
    right_on="gene_id",
    how="left"
)

# Keep only gene name + total intron length
result = merged[["GeneSymbol", "total_intron_length"]]

# Save for downstream analysis
result.to_csv("data/codon97_intron.csv", index=False)
print(result.head())

In [ ]:
# length for rna level
rna_df = pd.read_csv("data/RNA_Proteomics_RNAlevel.csv")

rna_df["ENSG_clean"] = rna_df["ESGN"].str.replace(r"\.\d+$", "", regex=True)

rna_with_introns = rna_df.merge(
    intron_total,
    left_on="ENSG_clean",   # make sure this matches your RNA file’s column name
    right_on="gene_id",
    how="left"
)

result = rna_with_introns[["GeneSymbol", "total_intron_length"]]

result.to_csv("data/rna_intron.csv", index=False)

In [ ]:
# length for proteomics
prot_df = pd.read_csv("data/RNA_Proteomics_Proteinlevel.csv")

prot_df["ENSG_clean"] = prot_df["ESGN"].str.replace(r"\.\d+$", "", regex=True)

prot_with_introns = prot_df.merge(
    intron_total,
    left_on="ENSG_clean",   # make sure this matches your Protein file’s column name
    right_on="gene_id",
    how="left"
)

result = prot_with_introns[["GeneSymbol", "total_intron_length"]]

result.to_csv("data/protein_intron.csv", index=False)

In [ ]:
# length for all_prot

all_df = pd.read_csv("data/RNA_Proteomics_Merged.csv")

all_df["ENSG_clean"] = all_df["Unnamed: 0"].str.replace(r"\.\d+$", "", regex=True)

all_with_introns = all_df.merge(
    intron_total,
    left_on="ENSG_clean",  
    right_on="gene_id",
    how="left"
)

all_gene_lengths = (
    all_with_introns.groupby("GeneSymbol")["total_intron_length"]
    .sum()
    .reset_index()
)

all_gene_lengths.to_csv("data/all_intron.csv", index=False)

#### bootstrap

In [ ]:
import pandas as pd
import numpy as np

codon97_df = pd.read_csv("/mnt/work_3/sijin/CAI/codon97_intron.csv").rename(columns={"total_intron_length": "Length"})
rna_df     = pd.read_csv("/mnt/work_3/sijin/CAI/rna_intron.csv").rename(columns={"total_intron_length": "Length"})
prot_df    = pd.read_csv("/mnt/work_3/sijin/CAI/protein_intron.csv").rename(columns={"total_intron_length": "Length"})
all_df     = pd.read_csv("/mnt/work_3/sijin/CAI/all_intron.csv").rename(columns={"total_intron_length": "Length"})

codon97_df["Source"] = "Codon97"
rna_df["Source"]     = "RNAseq_Log2FC>0"    
prot_df["Source"]    = "Proteomics_Log2FC>0"
all_df["Source"]     = "All_otherprot_all_Log2FC"

all_data = pd.concat([codon97_df, rna_df, prot_df, all_df], ignore_index=True)
all_data["Length"] = pd.to_numeric(all_data["Length"], errors="coerce")
all_data = all_data.dropna(subset=["Length"])
all_data = all_data[all_data["Length"] > 0] 

all_data["log10_Length"] = np.log10(all_data["Length"].astype(float))

def bootstrap_pvalue_mean(test_vals, all_vals, n_iter=10000, seed=123):
    np.random.seed(seed)
    test_mean = np.mean(test_vals)
    n = len(test_vals)

    boot_means = []
    for _ in range(n_iter):
        sample = np.random.choice(all_vals, size=n, replace=False)
        boot_means.append(np.mean(sample))

    boot_means = np.array(boot_means)
    all_mean = np.mean(all_vals)

    # Two-sided p-value around the All mean
    pval = np.mean(np.abs(boot_means - all_mean) >= np.abs(test_mean - all_mean))
    return test_mean, all_mean, pval

all_log = all_data.loc[all_data["Source"] == "All_otherprot_all_Log2FC", "log10_Length"].values

codon97_log = all_data.loc[all_data["Source"] == "Codon97", "log10_Length"].values
rna_log     = all_data.loc[all_data["Source"] == "RNAseq_Log2FC>0", "log10_Length"].values
prot_log    = all_data.loc[all_data["Source"] == "Proteomics_Log2FC>0", "log10_Length"].values

codon97_mean, all_mean1, p_cod_all = bootstrap_pvalue_mean(codon97_log, all_log)
rna_mean,     all_mean2, p_rna_all = bootstrap_pvalue_mean(rna_log, all_log)
prot_mean,    all_mean3, p_prot_all = bootstrap_pvalue_mean(prot_log, all_log)

res_df = pd.DataFrame({
    "Group": ["Codon97", "RNAseq_Log2FC>0", "Proteomics_Log2FC>0"],
    "Mean_log10_Group": [codon97_mean, rna_mean, prot_mean],
    "Mean_log10_All": [all_mean1, all_mean2, all_mean3],
    "p-value (two-sided)": [p_cod_all, p_rna_all, p_prot_all]
})

print("\n=== Bootstrap mean test on log10(total intron length): each group vs All_otherprot_all_Log2FC ===")
print(res_df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib as mpl

order = ["Codon97", "RNAseq_Log2FC>0", "Proteomics_Log2FC>0", "All_otherprot_all_Log2FC"]
counts = all_data.groupby("Source").size()

custom_palette = {
    "Codon97": "#8DD3C7",
    "RNAseq_Log2FC>0": "#c2bed6",
    "Proteomics_Log2FC>0": "#f6f6bc",
    "All_otherprot_all_Log2FC": "#eab375"
}

plt.figure(figsize=(10, 6))
ax = sns.violinplot(
    x="Source", y="log10_Length", data=all_data,
    inner=None, palette=custom_palette, order=order,
    linewidth=1.2, cut=0
)

sns.stripplot(
    x="Source", y="log10_Length", data=all_data,
    order=order, color="black", alpha=0.2,
    jitter=0.25, size=1.5
)

sns.boxplot(
    x="Source", y="log10_Length", data=all_data,
    order=order, width=0.15, showcaps=True,
    boxprops={"facecolor": "grey", "edgecolor": "black"},
    whiskerprops={"color": "black"},
    medianprops={"color": "black", "linewidth": 1},
    showfliers=False
)

group_means = all_data.groupby("Source")["log10_Length"].mean()
for i, src in enumerate(order):
    ax.scatter(i, group_means[src], color="white", edgecolors="black", zorder=3, s=30)

y_max = all_data["log10_Length"].max()

def add_bracket(x1, x2, y, h, text, fs=10):
    plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], c="k", lw=1.2)
    plt.text((x1+x2)/2, y+h+0.02, text, ha="center", va="bottom", fontsize=fs, color="black")

base_y = y_max + 0.10
step_y = 0.35
h = 0.10

add_bracket(0, 3, base_y + 0*step_y, h, f"p = {p_cod_all:.4f}*")
add_bracket(1, 3, base_y + 1*step_y, h, f"p = {p_rna_all:.4f}*")
add_bracket(2, 3, base_y + 2*step_y, h, f"p = {p_prot_all:.4f}")

ax.set_ylim(all_data["log10_Length"].min() - 0.1, base_y + 2*step_y + h + 0.25)

plt.xticks(
    ticks=range(len(order)),
    labels=[f"{src}\n(n={counts.get(src, 0)})" for src in order]
)

plt.title("Distribution of Total Intron Length (log10-transformed)", fontsize=14)
plt.ylabel("log10(Total Intron Length)", fontsize=12)
plt.xlabel("")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()
